# MSDS 696: Practicum II
## Emma Highland

## Introduction
This project uses the [MovieLens 100k Dataset](https://grouplens.org/datasets/movielens/100k/). Below I have included the citation for this data set.

> F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets:
History and Context. ACM Transactions on Interactive Intelligent
Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages.
DOI=http://dx.doi.org/10.1145/2827872

For this project, I performed exploratory data analysis and data cleaning in R (see associated .Rmd file). One step I took was to limit the genres included, using only genres that appeared a minimum of 10,000 times. Because individual movies are classified into multiple genres, this did not eliminate any movies. I chose to eliminate some more rare genres in order to obtain unique results, as I planned to follow a tutorial that used all data.

In this notebook, I use the K-Nearest Neighbors algorithm for two different purposes. First, I implement a KNN approach to suggest movies based on a given movie as a reference point. For this step, I follow the example of M. Hendra Herviawan, available for your consideration at [this link](https://hendra-herviawan.github.io/Movie-Recommendation-based-on-KNN-K-Nearest-Neighbors.html). Second, I implement a KNN approach using the library scikit-learn. The purpose of this second use of KNN was to predict genre from ratings. 

Overall, my goal is to examine the recommendations and consider how the different features of the dataset might inform the recommendations.

## Import libraries

In [268]:
# Import needed libraries
import pandas as pd
import numpy as np
from scipy import spatial
import operator
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

## Import and manipulate data

In [182]:
# View the first 15 rows of the imported Pandas data frame
movieDF = pd.read_csv('movie_new.csv', encoding='latin-1',usecols=range(1,11))
movieDF.head(n=15)

,Movie.ID,Movie.Title,Rating,Action,Adventure,Comedy,Drama,Romance,Sci-Fi,Thriller
0,242,Kolya (1996),3,0,0,1,0,0,0,0
1,302,L.A. Confidential (1997),3,0,0,0,0,0,0,1
2,377,Heavyweights (1994),1,0,0,1,0,0,0,0
3,51,Legends of the Fall (1994),2,0,0,0,1,1,0,0
4,346,Jackie Brown (1997),1,0,0,0,1,0,0,0
5,474,Dr. Strangelove or: How I Learned to Stop Worr...,4,0,0,0,0,0,1,0
6,265,"Hunt for Red October, The (1990)",2,1,0,0,0,0,0,1
7,465,"Jungle Book, The (1994)",5,0,1,0,0,1,0,0
8,451,Grease (1978),3,0,0,1,0,1,0,0
9,86,"Remains of the Day, The (1993)",3,0,0,0,1,0,0,0


In [160]:
# Following the example of M. Hendra Herviawan
# https://hendra-herviawan.github.io/Movie-Recommendation-based-on-KNN-K-Nearest-Neighbors.html
# Comments by me

movieStats = movieDF.groupby('Movie.ID').agg({'Rating': [np.size, np.mean]})
''' 
Group the movies according to ID (movies appear multiple times in data), then
aggregate the 'Rating' feature according to both the "size" (i.e. amount of
occurences) and the mean rating. Aggregation is done using numpy.
'''
movieStats.head() # View the first 10 rows

Rating          
           size      mean
Movie.ID                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

In [250]:
# Following the example of M. Hendra Herviawan
# https://hendra-herviawan.github.io/Movie-Recommendation-based-on-KNN-K-Nearest-Neighbors.html
# Comments by me

m = pd.DataFrame(movieStats['Rating']['size'])
# Save the size feature to a new variable, m
mNorm = m.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
''' 
Normalize the sizes to represent relative abudance of each movie.
The scale is 0 to 1. As worded by Herviawan, "a value of 0 means 
nobody rated it, and a value of 1 will mean it's the most popular 
movie there is."
Normalization is accomplished via applying lambda function that 
uses the normalization formula on the items in m.
'''
mNorm.head() # View the first 10 rows

,size
Movie.ID,
1,0.774914
2,0.223368
3,0.152921
4,0.357388
5,0.146048


In [190]:
len(movieDF)

100000

## Create a dictionary 

Now that the data has been manipulated, I will use the normalized data and the original Pandas data frame to create a dictionary. Because I had already done data cleaning in R, I had to follow a different procedure to create the dictionary than that presented by Herviawan. That said, the idea is the same. I created a dictionary that uses the movie ID as a key and that movie's title, genre list, relative abundance, and mean rating as values.

In [165]:
movieDict = {}
for i in range(len(movieDF)):
    name = movieDF['Movie.Title'][i]
    ID = movieDF['Movie.ID'][i]
    genre = [movieDF[x][i] for x in ['Action','Adventure','Comedy','Drama','Romance','Sci-Fi','Thriller']]
    movieDict[ID] = (name, genre, mNorm.loc[ID].get('size'), movieStats.loc[ID].Rating.get('mean'))

In [178]:
print(movieDict[1])
# The movie with the ID 1 is Toy Story

('Toy Story (1995)', [0, 0, 1, 0, 0, 0, 0], 0.7749140893470791, 3.8783185840707963)


In [183]:
print(movieDict[257])
# The movie with the ID 257 is Men in Black

('Men in Black (1997)', [1, 1, 1, 0, 0, 1, 0], 0.5189003436426117, 3.745874587458746)


## KNN for Recommendations

### ComputeDistance() Function

I used the following two functions from Herviawan's tutorial. The first computes distance between two movies based on genre - using cosine similarity - and the difference in relative abundance. The second computes the 10 nearest neighbors using the previous function. I have added commentary to both functions.

In [249]:
# Code from M. Hendra Herviawan
# https://hendra-herviawan.github.io/Movie-Recommendation-based-on-KNN-K-Nearest-Neighbors.html
# Comments by me

def ComputeDistance(a, b):
    genresA = a[1] # grab the genre list from the first input movie
    genresB = b[1] # grab the genre list from the second input movie
    genreDistance = spatial.distance.cosine(genresA, genresB)
    '''
    Compute the cosine similarity to find distance between two movies.
    Cosine similarity finds distance via computing the cosine of the
    angle between two vectors. Here, those two vectors are the lists 
    of genres. Although the genre lists are of the same length, cosine
    similarity is effective for vectors of different sizes as well.
    '''
    popularityA = a[2] # grab the popularity/relative abundance from a
    popularityB = b[2] # grab this information from b
    popularityDistance = abs(popularityA - popularityB)
    # Take the absolute value of the difference between a and b
    return genreDistance + popularityDistance
    # Return the distance as defined by both genre and relative abundance  

In [248]:
ComputeDistance(movieDict[1], movieDict[257])
# Compute the distance between Toy Story and Men in Black

0.7560137457044673

As noted by Herviawan, "the higher the distance, the less similar the movies are" and I would classify 0.75 to mean the movies are moderately different. I chose these two movies because they are both popular, family-friendly, action or adventure movies of the 1990s that launched franchises. I want to quickly remind the reader that a correlation chart was produced with R's PredictiveAnalytics library in the data exploration and cleaning phase. The highest correlation exists between action and adventure. Thus, the link between action and adventure movies is supported by the data.

Nota bene that some of my proposed similarities are not actually represented by the data. The algorithm *does not* consider that they came out only two years apart or care about their cultural significance. The algorithm *does* take into account the popularity (as defined by relative abundance) and genre. In this case, my concept of the genres for Toy Story would be adventure and comedy. However, it is placed only in the comedy category here. It would also belong in the 'Children' and 'Animation' categories, and does in the full dataset. It will be interesting to see what difference narrowing the genres makes. Although Toy Story is shown to be a popular movie, there is more limited data to distinguish it from other movies. Of course, the absence of categories could also distinguish it if most movies are still in multiple categories. Men in Black is placed in both, which is how I could classify it. It is also counted as Sci-Fi and a Comedy, categorizations with which I agree. Men in Black could very well be easier to find recomendations for than Toy Story.

Next, a function for KNN will be defined.

### getNeighbors() Function

In [244]:
# Code from M. Hendra Herviawan
# https://hendra-herviawan.github.io/Movie-Recommendation-based-on-KNN-K-Nearest-Neighbors.html
# Comments by me
def getNeighbors(movieID, K):
    distances = [] # Empty list to hold the distances
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    ''' 
    This for loop iterates through each key in movieDict.
    In each iteration, the key is assigned to the variable 'movie'
    If 'movie' is not being compared to itself (movie != movieID),
    the cosine similarity and popularity difference are computed.
    This means every movie is compared to every other movie. 
    At each comparison, the distance is assigned to the variable 'dist'
    and the empty list 'distances' stores this variable.
    '''
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors
'''
Sort the distances (index of 1 in the 'distances' list) in 
descending order (the default) and then define an empty list
for the neighbors. Then populate the empty list with the first
10 items in 'distances'.
'''

('Toy Story (1995)', [0, 0, 1, 0, 0, 0, 0], 0.7749140893470791, 3.8783185840707963) 



/home/emma/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Sting, The (1973) 4.058091286307054
Aladdin (1992) 3.8127853881278537
Mary Poppins (1964) 3.7247191011235956
Father of the Bride Part II (1995) 2.8984375
Kolya (1996) 3.9914529914529915
Romy and Michele's High School Reunion (1997) 3.061224489795918
Cool Runnings (1993) 3.161764705882353
To Wong Foo, Thanks for Everything! Julie Newmar (1995) 2.8947368421052633
Sleepless in Seattle (1993) 3.539906103286385
Bio-Dome (1996) 1.903225806451613


### KNN Implementation

In [251]:
# Code from M. Hendra Herviawan
# https://hendra-herviawan.github.io/Movie-Recommendation-based-on-KNN-K-Nearest-Neighbors.html
# Comments by me

K = 10 # 10 neighbors
avgRatingTS = 0 # start this variable at zero

print(movieDict[1], '\n') # Print the movie infomation using movie ID
neighbors = getNeighbors(1, K) # Get 10 neighbors for Toy Story (1995) 
for neighbor in neighbors:
    avgRatingTS += movieDict[neighbor][3]
    # Add the rating of the item to avgRatingTS each iteration
    print (movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))
    # Print the title, plus a space, plus the rating from movieDict

avgRatingTS /= K # Update avgRatingTS to be divided by K (10)

('Toy Story (1995)', [0, 0, 1, 0, 0, 0, 0], 0.7749140893470791, 3.8783185840707963) 

Sting, The (1973) 4.058091286307054
Aladdin (1992) 3.8127853881278537
Mary Poppins (1964) 3.7247191011235956
Father of the Bride Part II (1995) 2.8984375
Kolya (1996) 3.9914529914529915
Romy and Michele's High School Reunion (1997) 3.061224489795918
Cool Runnings (1993) 3.161764705882353
To Wong Foo, Thanks for Everything! Julie Newmar (1995) 2.8947368421052633
Sleepless in Seattle (1993) 3.539906103286385
Bio-Dome (1996) 1.903225806451613


/home/emma/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


The top ten related movies, according to this approach, are listed above. Some of them seem reasonable, some are not. The top movie is The Sting. I looked this movie up and have included a brief summary below, taken from [this Wikipedia article](https://en.wikipedia.org/wiki/The_Sting):
>The Sting is a 1973 American caper film set in September 1936, involving a complicated plot by two professional grifters (Paul Newman and Robert Redford) to con a mob boss (Robert Shaw)

This film is not at all like Toy Story in plot, but they share enough other traits that it was recommended. This is a good example of code doing what you tell it to do, but not what you want it to do. Of course, some of the suggestions are reasonable, such as Aladdin and Mary Poppins.

Of note is the fact that in Herviawan's tutorial an almost completely different list was generated. The only common feature is Aladdin, which is the second nearest neighbor in both. Herviawan's result was:
* Liar Liar (1997) 3.15670103093
* Aladdin (1992) 3.81278538813
* Willy Wonka and the Chocolate Factory (1971) 3.63190184049
* Monty Python and the Holy Grail (1974) 4.0664556962
* Full Monty, The (1997) 3.92698412698
* George of the Jungle (1997) 2.68518518519
* Beavis and Butt-head Do America (1996) 2.78846153846
* Birdcage, The (1996) 3.44368600683
* Home Alone (1990) 3.08759124088
* Aladdin and the King of Thieves (1996) 2.84615384615

It appears that removing the less common genres had a big effect on the recommendation.


Next, I will continue to follow the tutorial's example and compare the average rating I computed for the 10 neighbors as well as the actual film's overall average rating. Comparing the two ratings will help me evaluate the performance of the KNN implementation. In the tutorial, the average rating of the neighbors was ~3.3446.

In [245]:
avgRatingTS

3.304634421453303

Although the list of movies was very different, the average rating only differs by about 0.04 points. For comparison, the average rating of the film is as follows:

In [246]:
movieDict[1][3]

3.8783185840707963

The difference is by ~0.57 points. Not too bad, but not outstanding.

I am going to repeat this process for Men in Black. This is a fresh example. Recall, Toy Story and Men in Black were moderately different in my reading of the ComputeDistance() function's output. The number was ~0.756. That said, I personally think the movies have some shared qualities. I am interested to see if there will be any overlap between the two lists of recommendations.

In [252]:
K= 10
avgRatingMiB = 0

print(movieDict[257], '\n')
neighbors = getNeighbors(257, K) # Men in Black (1997)
for neighbor in neighbors:
    avgRatingMiB += movieDict[neighbor][3]
    print (movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))

avgRatingMiB /= K

('Men in Black (1997)', [1, 1, 1, 0, 0, 1, 0], 0.5189003436426117, 3.745874587458746) 

Star Trek: First Contact (1996) 3.66027397260274
Mission: Impossible (1996) 3.313953488372093
Batman Returns (1992) 2.683098591549296
True Lies (1994) 3.5625
Terminator, The (1984) 3.9335548172757475
Twister (1996) 3.2150170648464163
Batman Forever (1995) 2.6666666666666665
Jumanji (1995) 3.3125
Evil Dead II (1987) 3.5168539325842696
Abyss, The (1989) 3.589403973509934


/home/emma/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


There is no overlap, but the suggestions here are a lot more reasonable. The results here include other Sci-Fi movies, other Adventure movies, and other Action movies. Although Evil Dead II may sound out of place, it is actually not a bad recommendation for someone who likes Men in Black. Of course, I am biased because Evil Dead II is coincidentally my favorite movie.

In [241]:
avgRatingMiB

3.345382250740716

In [243]:
movieDict[257][3]

3.745874587458746

This time, the numbers are a bit closer, off by 0.4 instead of about 0.57. I interpret this result to mean that the KNN implementation performed slightly better for Men in Black. This may have to do with Men in Black's relative abundance and/or to do with it being in more categories. With a relative abundance/popularity score of ~0.52, it's essentially right in the middle in terms of popularity. Toy Story was more prevalent, at ~0.77.

## KNN for Classifications

### Create Numpy arrays

Having used KNN for recommendations, I want to explore a second way to use the algorithm on this dataset. Instead of recommending movies, I am going to try to predict the genre of a movie based on the rating. This time, I am going to use the scikit-learn library's version of KNN. If it is easy to predict rating from genre, I would conclude that there is a strong link between the two. If it is not easy, I would conclude there is a weak link. I am curious about this because both were contributors to the recommendations provided above. The correlation chart I previously mentioned showed that all genres were significantly correlated, with Action/Adventure in the lead. Correlations within a dataset can be informative. They can also be a problem for machine learning with some approaches, as seen by Linear Regression's undoing in the presence of multicollinearity. Thankfully, this does not apply to KNN classification. Regardless, I seek more information about how genre and rating might correlate.

First, I have to do some data preparation. I am going to build two numpy arrays, one with genres and one with rounded ratings (i.e. 1, 2, 3, 4, or 5 stars). Then I will output the first 10 items of each, in addition to the dimensions.

In [227]:
myGenres = np.array([movieDict[i][1] for i in movieDict.keys()])
myRating = np.array([[round(movieDict[i][3])] for i in movieDict.keys()])

In [228]:
myGenres[:10]

array([[0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0]])

In [229]:
myGenres.shape

(1682, 7)

In [230]:
myRating[:10]

array([[4.],
       [4.],
       [2.],
       [3.],
       [4.],
       [4.],
       [4.],
       [4.],
       [3.],
       [4.]])

In [231]:
myRating.shape

(1682, 1)

### Train/Test Split

In [277]:
''' 
The parameters for test size and random_state that are used
here were copied from documentation.
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
The split is 25% testing, 75% training. 
The random_state is the seed, which is set for reproducibility.
'''
X_train, X_test, y_train, y_test = train_test_split(myRating, myGenres, test_size=0.25, random_state=333)

### Fit KNN classifier

In [278]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

### KNN Predictions

As a reminder, the categories for genre are in this order: Action, Adventure, Comedy, Drama, Romance, Sci-Fi, and Thriller.

I will generate predictions for 1, 2, 3, 4, and 5 stars. This was the purpose of rounding the ratings earlier.

In [279]:
print(knn.predict([[1.]]))

[[0 0 0 1 0 0 0]]


In [280]:
print(knn.predict([[2.]]))

[[0 0 0 0 0 0 0]]


In [281]:
print(knn.predict([[3.]]))

[[0 0 0 0 0 0 0]]


In [282]:
print(knn.predict([[4.]]))

[[0 0 0 1 0 0 0]]


In [283]:
print(knn.predict([[5.]]))

[[0 0 0 1 0 0 0]]


The only ratings that generated predictions were 1 star, 4 stars, and 5 stars, all of which predict comedy. Ratings of 2 and 3 suggest no genres.

### Scoring the KNN model

Having trained the KNN model on the training set, it's time to evaluate the model using the testing set.

In [276]:
knn.score(X_test,y_test)

0.21852731591448932

The mean accuracy for the KNN model is 21.8%. This result indicates that there is not a strong link between rating and genre, at least not one detectable by a KNN approach. I would therefore conclude that genre and rating are not jointly influencing the recommendations, but instead are distinct influences.

## Conclusion

My goal was to examine the recommendations and consider how the different features of the dataset might inform the recommendations.

My major take aways from this analysis are:
* The tutorial KNN implementation is fairly successful as is
* The amount of genres used in the dataset has a larger influence on which movies are recommended than the ratings do
* Rating and genre do not have a strong relationship